In [3]:
import pandas as pd
import mysql.connector
import re

import json

from IPython.display import display

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo",
    charset="latin1"
)

subjects = [
    #'Psychologie',
    #'English for Spanish Natives',
    #'Medienbildung',
    'Biologie',
    #'Kommunikation',
    'Wirtschaft',
    'Mathe',
    'Chemie',
    #'Geschichte',
    'Lerntipps',
    'Informatik',
    #'Musik',
    #'Testbereich',
    #'Deutsch als Fremdsprache',
    #'Englisch',
    #'Betriebswirtschaftslehre mit Rechnungswesen',
    'Geographie',
    #'Rechnungswesen',
    'Nachhaltigkeit',
    #'Politik',
    'Physik',
    #'Community',
    #'Latein'
]

def cached(func):
    cache = dict()
    
    def return_func(arg):
        if (arg in cache):
            return cache[arg]
        else:
            result = func(arg)
            cache[arg] = result
            return result
    
    return return_func

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def querySingleton(sql):
    return [ x[0] for x in query(sql) ]

@cached
def getParent(termId):
    return querySingleton("""
        select parent_id from term_taxonomy where id = %s;
    """ % termId)[0]

def getTermName(termId):
    return querySingleton("""
        select term.name from term_taxonomy
        join term on term.id = term_taxonomy.term_id
        where term_taxonomy.id = %s;
    """ % termId)[0]

@cached
def getSubject(termId):
    parent = getParent(termId)
    grandparent = getParent(parent)
    
    if (parent == 106081):
        return getTermName(termId)
    
    return getSubject(parent) if grandparent != None else getTermName(termId)

def getSerloKeywords():
    keywords = pd.read_sql("""
        select term_taxonomy_entity.term_taxonomy_id as termId,
               entity_revision_field.value as title
        from entity
        join entity_revision_field on entity_revision_field.entity_revision_id = entity.current_revision_id
        join type on type.id = entity.type_id
        join uuid on uuid.id = entity.id
        join term_taxonomy_entity on term_taxonomy_entity.entity_id  = entity.id
        where entity.instance_id = 1
            and uuid.trashed = 0
            and (type.name = "article" or type.name = "course")
            and (entity_revision_field.field = "title" or entity_revision_field.field = "meta_title");
    """, db)
    keywords["subject"] = keywords["termId"].apply(getSubject)
    keywords.drop(keywords[~keywords["subject"].isin(subjects)].index, inplace=True)
    
    keywords = set((normalize(keyword) for keyword in keywords["title"]))
    
    return [keyword for keyword in keywords if len(keyword) > 0]

def normalize(title):
    title = bytes(title, "latin1").decode("utf8")
    title = title.replace(" - Lernen mit Serlo!", "").lower()
    
    words = set([word for word in re.split("\W+", title) if isKeyword(word)])
    words = list(words)
    words.sort()
    
    if (len(words) > 1):
        return ""
    
    return " ".join(words)

words = set()

def isKeyword(word):
    if len(word) <= 3 and word not in ["web", "key", "dna", "ggt"]:
        words.add(word)
        return False
    if word in ["eine", "kurs", "eines", "einer", "oder", "sind", "test"]:
        return False
    return True

keywords = getSerloKeywords()

with open("/tmp/keywords.json", "w") as fd:
    json.dump(keywords, fd)

display(len(keywords))
display(keywords)

485

['zwischenwertsatz',
 'wurzelfunktion',
 'sekante',
 'körpernetze',
 'kerzen',
 'schallgeschwindigkeit',
 'bilanz',
 'wertemenge',
 'mesomerie',
 'milchprodukte',
 'aggregatzustand',
 'bogenmaß',
 'lunge',
 'biodiversität',
 'halogene',
 'cluster',
 'polyester',
 'nullstelle',
 'elektronegativität',
 'bakterieninfektionen',
 'kreiszahl',
 'edelgaszustand',
 'drachenviereck',
 'wurzel',
 'klima',
 'ringsöffnungspolymerisation',
 'veresterung',
 'klimawandel',
 'faktorisieren',
 'datentypen',
 'potenzen',
 'wurmfarm',
 'kongruenz',
 'mischkultur',
 'prozent',
 'kettenregel',
 'funktionenschar',
 'attribute',
 'geldeinheiten',
 'binomialverteilung',
 'hypothesentest',
 'evolutionsfaktoren',
 'additionssysteme',
 'stochastik',
 'skalar',
 'umkehrfunktion',
 'methode',
 'einkommensteuer',
 'erntetricks',
 'chromosomenmutation',
 'internetabhängigkeit',
 'immunisierung',
 'logarithmen',
 'basenkonstante',
 'normalverteilung',
 'strecke',
 'fasskreisbogen',
 'alkine',
 'hochbeet',
 'volumen',